<a href="https://www.kaggle.com/code/saugatkafley/binary-machine-failures-randomforest-kfold?scriptVersionId=134773565" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e17/sample_submission.csv
/kaggle/input/playground-series-s3e17/train.csv
/kaggle/input/playground-series-s3e17/test.csv
/kaggle/input/machine-failure-predictions/machine failure.csv


# Dataset 

In [2]:
sample = pd.read_csv("/kaggle/input/playground-series-s3e17/sample_submission.csv")
train = pd.read_csv("/kaggle/input/playground-series-s3e17/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s3e17/test.csv")
original = pd.read_csv("/kaggle/input/machine-failure-predictions/machine failure.csv")

print('The dimension of the train synthetic dataset is:', train.shape)
print('The dimension of the test synthetic dataset is:', test.shape)
print('The dimension of the orginal dataset is:', original.shape)
print('The dimension of the submission dataset is:', sample.shape)


The dimension of the train synthetic dataset is: (136429, 14)
The dimension of the test synthetic dataset is: (90954, 13)
The dimension of the orginal dataset is: (10000, 14)
The dimension of the submission dataset is: (90954, 2)


In [3]:
train.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [4]:
sample.head()

,id,Machine failure
0,136429,0.5
1,136430,0.5
2,136431,0.5
3,136432,0.5
4,136433,0.5


In [5]:
train.dtypes == object

id                         False
Product ID                  True
Type                        True
Air temperature [K]        False
Process temperature [K]    False
Rotational speed [rpm]     False
Torque [Nm]                False
Tool wear [min]            False
Machine failure            False
TWF                        False
HDF                        False
PWF                        False
OSF                        False
RNF                        False
dtype: bool

In [6]:
train.describe()

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
mean,68214.000000,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,136428.000000,304.400000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
train.Type.unique()

array(['L', 'M', 'H'], dtype=object)

In [8]:
train["Type"] = train.Type.map( {"L":0 , "M":1 , "H":2})
train.Type

0         0
1         1
2         0
3         0
4         1
         ..
136424    1
136425    2
136426    0
136427    0
136428    0
Name: Type, Length: 136429, dtype: int64

In [9]:
test["Type"] = test.Type.map( {"L":0 , "M":1 , "H":2})
test.Type

0        0
1        0
2        0
3        1
4        1
        ..
90949    0
90950    0
90951    0
90952    0
90953    0
Name: Type, Length: 90954, dtype: int64

In [10]:
train["Machine failure"].unique()

array([0, 1])

In [11]:
train.isna().sum()

id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

In [12]:
test.shape

(90954, 13)

## Preprocessing

In [13]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler , RobustScaler


In [14]:
y_train = train[["Machine failure"]]
y_train.head()

,Machine failure
0,0
1,0
2,0
3,0
4,0


In [15]:
cols = train.columns.to_list()
cols.append(cols.pop(8))

In [16]:
train = train[cols]
test = test[cols[:-1]]

In [17]:
X_train = train.iloc[:,2:-1]
y_train = train.iloc[:, -1]

In [18]:
X_train.columns

Index(['Type', 'Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'TWF',
       'HDF', 'PWF', 'OSF', 'RNF'],
      dtype='object')

In [19]:
sc = StandardScaler()
mms = MinMaxScaler()
rbs = RobustScaler()
X_train.iloc[:,:] = rbs.fit_transform(X_train.iloc[:,:])
X_train.head()

/tmp/ipykernel_29/1133549392.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train.iloc[:,:] = rbs.fit_transform(X_train.iloc[:,:])


,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,0.0,0.206897,-0.181818,0.695946,-0.373913,0.306306,0.0,0.0,0.0,0.0,0.0
1,1.0,0.896552,0.954545,1.797297,-0.982609,0.846847,0.0,0.0,0.0,0.0,0.0
2,0.0,-0.241379,-0.681818,2.108108,-1.208696,-0.729730,0.0,0.0,0.0,0.0,0.0
3,0.0,0.344828,0.409091,0.209459,0.339130,0.819820,0.0,0.0,0.0,0.0,0.0
4,1.0,-0.689655,-0.454545,1.000000,-0.434783,-0.648649,0.0,0.0,0.0,0.0,0.0


In [20]:
X_train.shape

(136429, 11)

In [21]:
X_test = test.iloc[:, 2:]
X_test.shape

(90954, 11)

In [22]:
X_test.dtypes

Type                         int64
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64
dtype: object

In [23]:
X_test.iloc[:,:] = rbs.transform(X_test.iloc[:,:])
# X_test.iloc[:,:] = mms.transform(X_test.iloc[:,:])

X_test.head()

/tmp/ipykernel_29/3900378882.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_test.iloc[:,:] = rbs.transform(X_test.iloc[:,:])


,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,0.0,0.793103,0.681818,0.040541,-0.208696,-0.414414,0.0,0.0,0.0,0.0,0.0
1,0.0,0.586207,0.454545,1.486486,-1.008696,-0.801802,0.0,0.0,0.0,0.0,0.0
2,0.0,0.448276,0.181818,0.216216,-0.234783,-0.090090,0.0,0.0,0.0,0.0,0.0
3,1.0,0.034483,-0.181818,-0.094595,0.626087,-0.909910,0.0,0.0,0.0,0.0,0.0
4,1.0,1.172414,1.045455,0.148649,0.078261,0.072072,0.0,0.0,0.0,0.0,0.0


## Decision Tree Classifier

In [24]:
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss


In [25]:
X_train.shape , y_train.shape

((136429, 11), (136429,))

In [26]:
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold , cross_val_score , GridSearchCV

# model = CatBoostClassifier(
#     iterations = 10000,
#     learning_rate = 1e-4,
#     l2_leaf_reg=4.0,
#     random_seed = 43,
#     metric_period = 200,).fit(X_train, y_train)
# val_accuracy = best_model.score(, y_val)

# X = X_train.copy()
# Y = y_train.copy()
# test_baseline = X_test.copy()

# cv_scores, preds = list(), list()

# skf = StratifiedKFold(n_splits = 10,  shuffle = True , random_state = 42)

# for i , (train_ix , test_ix) in enumerate(skf.split(X, Y)):
#     X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
#     Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
#     print('---------------------------------------------------------------')
#     model = CatBoostClassifier(
        
#         metric_period = 200, 
#         eval_metric = 'AUC',
#     ).fit(X_train , Y_train)
# #     model = RandomForestClassifier().fit(X_train , Y_train)
# #     gb_model = GradientBoostingClassifier(
# #         n_estimators = 500 , 
# #         max_depth = 7 , 
# #         learning_rate = 0.01, 
# #         min_samples_split = 10 , 
# #         min_samples_leaf = 20,
# #     ).fit(X_train, Y_train)
#     pred_1 = model.predict_proba(X_test)[:,1]
#     pred_2  =model.predict_proba(test_baseline)[:,1]

#     score_fold = roc_auc_score(Y_test , pred_1)
#     cv_scores.append(score_fold)
#     preds.append(pred_2)
#     print('Fold', i, '==> RandomForest oof ROC-AUC score is ==>', score_fold)

In [27]:
# model = XGBClassifier(
#     objective='binary:logistic', 
#     eval_metric='auc', 
#     random_state=43,
#     n_estimators = 900,
#     subsample=0.83,
#     colsample_bytree=0.17,
#     learning_rate=0.06,
#     max_depth=16,
# ).fit(X_train.values, y_train.values )


In [28]:
cb_parames = {
    'n_estimators': 1600,
    'max_depth': 165,
    'learning_rate': 0.00343900835845922,
    'max_bin': 295,
    'random_strength': 0.23,
    'grow_policy': 'Lossguide',
    'bootstrap_type': 'Bayesian',
    'objective':'Logloss',
    "loss_function": "AUC",
    'eval_metric': "AUC",
    'l2_leaf_reg':  0.06876484483245901,
    'min_child_samples': 185,
    'random_state': 13,
    'silent': True,
    'task_type': 'GPU',
}

# Build the model with the best parameters
model = CatBoostClassifier(**cb_parames).fit(X_train,y_train)

# preds = model.predict_proba(X_test)[:,-1]

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 138ms	remaining: 3m 34s
1:	total: 216ms	remaining: 2m 47s
2:	total: 366ms	remaining: 3m 8s
3:	total: 436ms	remaining: 2m 48s
4:	total: 509ms	remaining: 2m 37s
5:	total: 586ms	remaining: 2m 30s
6:	total: 667ms	remaining: 2m 27s
7:	total: 747ms	remaining: 2m 23s
8:	total: 920ms	remaining: 2m 37s
9:	total: 1.01s	remaining: 2m 34s
10:	total: 1.16s	remaining: 2m 41s
11:	total: 1.38s	remaining: 2m 56s
12:	total: 1.53s	remaining: 3m 1s
13:	total: 1.73s	remaining: 3m 9s
14:	total: 1.86s	remaining: 3m 10s
15:	total: 1.99s	remaining: 3m 10s
16:	total: 2.18s	remaining: 3m 16s
17:	total: 2.34s	remaining: 3m 18s
18:	total: 2.5s	remaining: 3m 21s
19:	total: 2.65s	remaining: 3m 22s
20:	total: 2.81s	remaining: 3m 24s
21:	total: 3.01s	remaining: 3m 28s
22:	total: 3.23s	remaining: 3m 34s
23:	total: 3.45s	remaining: 3m 39s
24:	total: 3.62s	remaining: 3m 40s
25:	total: 3.81s	remaining: 3m 43s
26:	total: 3.99s	remaining: 3m 45s
27:	total: 4.18s	remaining: 3m 47s
28:	total: 4.37s	remaining: 3m 49s

In [29]:
preds = model.predict_proba(X_test.values)[:,-1]

In [30]:
preds

array([0.00179768, 0.00188486, 0.00096153, ..., 0.00113462, 0.00088255,
       0.00156155])

In [31]:
roc_auc_score(y_train, model.predict_proba(X_train)[:,-1])

0.9836772742481417

In [32]:
# cv_score = np.max(cv_scores)
# cv_score

# cross_val_score(model, X_train, y_train , n_jobs = -1, verbose = 1 , scoring = 'roc_auc')

In [33]:
# np.mean(scores)

In [34]:
# from scipy import stats
# preds_test = pd.DataFrame(preds).apply(np.mean , axis = 0)

In [36]:
submission = pd.read_csv("/kaggle/input/playground-series-s3e17/sample_submission.csv")
submission['Machine failure'] = preds
submission.to_csv("/kaggle/working/submission.csv", index = False)